<a href="https://colab.research.google.com/github/nicolasyassuda/super-comp/blob/main/Aula12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
%%writefile ex1.cu

#include <thrust/device_vector.h>
#include <thrust/host_vector.h>
#include <thrust/transform.h>
#include <iostream>
#include <cstdlib>  // Para usar a função rand()
#include <ctime>    // Para a função time(), usada na inicialização da semente do rand()

// Functor para realizar o cálculo Saxpy
struct saxpy
{
    int a;
    saxpy(int a_) : a(a_) {}  // Construtor que inicializa a constante 'a'

    __host__ __device__
    double operator()(const int& x, const int& y) const {
        return a * x + y;  // Operação a⋅x + y
    }
};

int main() {
    // Inicializar a semente aleatória para gerar números diferentes em cada execução
    std::srand(std::time(0));

    // Tamanho dos vetores
    const int N = 10;

    for (int iteration = 0; iteration < 5; ++iteration) {
        // Gerar um valor aleatório para a constante 'a'
        int a = std::rand() % 10 + 1;  // Gera um valor aleatório entre 1 e 10

        // Vetores no host (CPU)
        thrust::host_vector<int> h_x(N);
        thrust::host_vector<int> h_y(N);

        // Preenche os vetores com valores aleatórios
        for(int i = 0; i < N; ++i) {
            h_x[i] = rand() % 100;  // Valores aleatórios de 0 a 99
            h_y[i] = rand() % 100;
        }

        // Copiar os vetores para o device (GPU)
        thrust::device_vector<int> d_x = h_x;
        thrust::device_vector<int> d_y = h_y;
        thrust::device_vector<int> d_z(N);  // Vetor para armazenar o resultado

        // Aplicar a operação saxpy com o valor aleatório de 'a'
        thrust::transform(d_x.begin(), d_x.end(), d_y.begin(), d_z.begin(), saxpy(a));

        // Copiar o resultado de volta para o host
        thrust::host_vector<int> h_z = d_z;

        // Exibir os resultados
        std::cout << "Execução " << iteration + 1 << " (a = " << a << "):\n";
        std::cout << "Resultado do cálculo Saxpy (a * x + y):\n";
        for(int i = 0; i < N; ++i) {
            std::cout << "a * " << h_x[i] << " + " << h_y[i] << " = " << h_z[i] << std::endl;
        }
        std::cout << "-----------------------------\n";
    }

    return 0;
}


Overwriting ex1.cu


In [18]:
!nvcc -arch=sm_70 -std=c++14 ex1.cu -o ex1

In [19]:
!./ex1

Execução 1 (a = 8):
Resultado do cálculo Saxpy (a * x + y):
a * 18 + 68 = 212
a * 89 + 81 = 793
a * 9 + 25 = 97
a * 52 + 35 = 451
a * 92 + 20 = 756
a * 95 + 83 = 843
a * 9 + 25 = 97
a * 71 + 21 = 589
a * 90 + 47 = 767
a * 79 + 42 = 674
-----------------------------
Execução 2 (a = 1):
Resultado do cálculo Saxpy (a * x + y):
a * 25 + 60 = 85
a * 48 + 51 = 99
a * 89 + 52 = 141
a * 64 + 69 = 133
a * 67 + 61 = 128
a * 39 + 88 = 127
a * 3 + 21 = 24
a * 97 + 80 = 177
a * 25 + 85 = 110
a * 73 + 46 = 119
-----------------------------
Execução 3 (a = 1):
Resultado do cálculo Saxpy (a * x + y):
a * 8 + 7 = 15
a * 5 + 31 = 36
a * 81 + 48 = 129
a * 78 + 60 = 138
a * 90 + 60 = 150
a * 85 + 3 = 88
a * 8 + 36 = 44
a * 44 + 13 = 57
a * 1 + 13 = 14
a * 80 + 14 = 94
-----------------------------
Execução 4 (a = 6):
Resultado do cálculo Saxpy (a * x + y):
a * 20 + 69 = 189
a * 26 + 18 = 174
a * 50 + 3 = 303
a * 3 + 23 = 41
a * 49 + 83 = 377
a * 31 + 57 = 243
a * 41 + 62 = 308
a * 38 + 89 = 317
a * 93 + 5

In [23]:
%%writefile ex2.cu

#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <cmath>  // Para a função std::sqrt()
#include <iostream>
#include <cstdlib>  // Para rand() e srand()
#include <ctime>    // Para time()

// Functor para elevar ao quadrado
struct square
{
    __host__ __device__
    float operator()(const float& x) const {
        return x * x;  // Eleva ao quadrado
    }
};

// Função que calcula a magnitude de um vetor de floats usando Thrust
float magnitude(thrust::device_vector<float>& v) {
    // thrust::transform_reduce aplica a transformação (square) e soma os resultados
    float sum_of_squares = thrust::transform_reduce(v.begin(), v.end(), square(), 0.0f, thrust::plus<float>());
    return std::sqrt(sum_of_squares);  // Retorna a raiz quadrada da soma dos quadrados
}

int main() {
    const int N = 1000000;  // Tamanho do vetor
    std::vector<float> h_v(N);  // Usando std::vector no host

    // Inicializar a semente para os valores aleatórios
    std::srand(std::time(0));

    // Preencher o vetor com valores aleatórios
    for(int i = 0; i < N; ++i) {
        h_v[i] = static_cast<float>(rand()) / RAND_MAX;  // Valores entre 0 e 1
    }

    // Copiar o vetor para a GPU (device)
    thrust::device_vector<float> d_v = h_v;  // Converte std::vector em device_vector

    // Medir o tempo de execução na GPU
    clock_t start_gpu = clock();
    float result_gpu = magnitude(d_v);
    clock_t end_gpu = clock();
    double gpu_time = double(end_gpu - start_gpu) / CLOCKS_PER_SEC;

    std::cout << "Magnitude (GPU): " << result_gpu << std::endl;
    std::cout << "Tempo de execução na GPU: " << gpu_time << " segundos" << std::endl;

    return 0;
}



Overwriting ex2.cu


In [24]:
!nvcc -arch=sm_70 -std=c++14 ex2.cu -o ex2

In [26]:
!./ex2

Magnitude (GPU): 577.049
Tempo de execução na GPU: 0.000852 segundos


In [27]:
%%writefile ex2-2.cpp

#include <iostream>
#include <vector>
#include <cmath>  // Para a função std::sqrt()
#include <cstdlib>  // Para rand() e srand()
#include <ctime>    // Para clock() e time()

// Função que calcula a magnitude de um vetor na CPU
float magnitude_cpu(const std::vector<float>& v) {
    float sum_of_squares = 0.0f;
    for(const auto& value : v) {
        sum_of_squares += value * value;
    }
    return std::sqrt(sum_of_squares);
}

int main() {
    const int N = 1000000;  // Tamanho do vetor
    std::vector<float> v(N);

    // Inicializar a semente para os valores aleatórios
    std::srand(std::time(0));

    // Preencher o vetor com valores aleatórios
    for(int i = 0; i < N; ++i) {
        v[i] = static_cast<float>(rand()) / RAND_MAX;  // Valores entre 0 e 1
    }

    // Medir o tempo de execução na CPU
    clock_t start_cpu = clock();
    float result_cpu = magnitude_cpu(v);
    clock_t end_cpu = clock();
    double cpu_time = double(end_cpu - start_cpu) / CLOCKS_PER_SEC;

    std::cout << "Magnitude (CPU): " << result_cpu << std::endl;
    std::cout << "Tempo de execução na CPU: " << cpu_time << " segundos" << std::endl;

    return 0;
}


Writing ex2-2.cpp


In [28]:
!g++ ex2-2.cpp -o ex2-2

In [29]:
!./ex2-2

Magnitude (CPU): 577.455
Tempo de execução na CPU: 0.014471 segundos


In [40]:
%%writefile ex3.cu

#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <iostream>
#include <cstdlib>
#include <cmath>  // Para sqrt() e fabs()
#include <ctime>  // Para clock()

// Functor para calcular a diferença ao quadrado
struct diff_square_op
{
    float mean;
    diff_square_op(float mean_) : mean(mean_) {}  // Construtor que armazena a média

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;  // Diferença em relação à média
        return diff * diff;     // Eleva ao quadrado
    }
};

// Functor para calcular a variância
struct variance_op
{
    float mean;
    variance_op(float mean_) : mean(mean_) {}  // Construtor que armazena a média

    __host__ __device__
    float operator()(const float& x) const {
        float diff = x - mean;  // Diferença em relação à média
        return diff * diff;     // Elevar ao quadrado a diferença
    }
};

// Função que calcula a variância usando fusion kernel
float calculate_variance_fusion(const thrust::device_vector<float>& d_vec, float mean) {
    // Usa transform_reduce para aplicar a operação variance_op e reduzir com a soma
    return thrust::transform_reduce(d_vec.begin(), d_vec.end(), variance_op(mean), 0.0f, thrust::plus<float>()) / d_vec.size();
}

// Função que calcula a variância em duas etapas
float calculate_variance_separate(const thrust::device_vector<float>& d_vec, float mean) {
    // Etapa 1: Subtrair a média de cada elemento e elevar ao quadrado
    thrust::device_vector<float> diff_squared(d_vec.size());
    thrust::transform(d_vec.begin(), d_vec.end(), diff_squared.begin(), diff_square_op(mean));

    // Etapa 2: Somar os quadrados das diferenças e dividir pelo tamanho do vetor
    float sum_of_squares = thrust::reduce(diff_squared.begin(), diff_squared.end(), 0.0f, thrust::plus<float>());
    return sum_of_squares / d_vec.size();
}

// Função que calcula a média
float calculate_mean(const thrust::device_vector<float>& d_vec) {
    float sum = thrust::reduce(d_vec.begin(), d_vec.end(), 0.0f, thrust::plus<float>());
    return sum / d_vec.size();
}

int main() {
    // Tamanhos dos vetores a serem testados
    std::vector<int> sizes = {100000, 500000, 1000000, 5000000};

    for (int size : sizes) {
        std::cout << "Tamanho do vetor: " << size << std::endl;

        // Gerar vetor aleatório no host
        std::vector<float> h_vec(size);
        std::srand(std::time(0));

        for(int i = 0; i < size; ++i) {
            h_vec[i] = static_cast<float>(rand()) / RAND_MAX;
        }

        // Copiar o vetor para o device
        thrust::device_vector<float> d_vec = h_vec;

        // Calcular a média (passo comum para ambas as abordagens)
        clock_t start = clock();
        float mean = calculate_mean(d_vec);
        clock_t end = clock();
        double mean_time = double(end - start) / CLOCKS_PER_SEC;

        std::cout << "Média calculada: " << mean << std::endl;

        // Parte 1: Variância usando Fusion Kernel
        start = clock();
        float variance_fusion = calculate_variance_fusion(d_vec, mean);
        end = clock();
        double fusion_time = double(end - start) / CLOCKS_PER_SEC;

        std::cout << "Variância (Fusion Kernel): " << variance_fusion << std::endl;
        std::cout << "Tempo de execução (Fusion Kernel): " << fusion_time << " segundos" << std::endl;

        // Parte 2: Variância usando abordagem de duas etapas
        start = clock();
        float variance_separate = calculate_variance_separate(d_vec, mean);
        end = clock();
        double separate_time = double(end - start) / CLOCKS_PER_SEC;

        std::cout << "Variância (Duas Etapas): " << variance_separate << std::endl;
        std::cout << "Tempo de execução (Duas Etapas): " << separate_time << " segundos" << std::endl;

        std::cout << "----------------------------------------" << std::endl;
    }

    return 0;
}



Overwriting ex3.cu


In [41]:
!nvcc -arch=sm_70 -std=c++14 ex3.cu -o ex3

In [42]:
!./ex3

Tamanho do vetor: 100000
Média calculada: 0.499581
Variância (Fusion Kernel): 0.0833519
Tempo de execução (Fusion Kernel): 8.5e-05 segundos
Variância (Duas Etapas): 0.0833519
Tempo de execução (Duas Etapas): 0.000148 segundos
----------------------------------------
Tamanho do vetor: 500000
Média calculada: 0.499872
Variância (Fusion Kernel): 0.0834915
Tempo de execução (Fusion Kernel): 5e-05 segundos
Variância (Duas Etapas): 0.0834915
Tempo de execução (Duas Etapas): 0.000265 segundos
----------------------------------------
Tamanho do vetor: 1000000
Média calculada: 0.500016
Variância (Fusion Kernel): 0.0833621
Tempo de execução (Fusion Kernel): 0.000213 segundos
Variância (Duas Etapas): 0.0833621
Tempo de execução (Duas Etapas): 0.000456 segundos
----------------------------------------
Tamanho do vetor: 5000000
Média calculada: 0.500092
Variância (Fusion Kernel): 0.0833179
Tempo de execução (Fusion Kernel): 0.000288 segundos
Variância (Duas Etapas): 0.0833179
Tempo de execução (Dua